In [628]:
import numpy as np  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import pandas as pd  # type: ignore

In [629]:
NUM_ITERATIONS = 10
NUM_WOLFS = 10

In [630]:
NUM_INTERVALS = 96

In [631]:
# Matriz de carros: 1-Modelo | 2-Capacidade (kM) | 3-ID | 4-disponibilidade | 5-viagem | 6-Capacidade | 7-Capacidade da bateria (kWh) | 8-Taxa de carga da bateria (kW) | 9-Taxa de descarga da bateria (kW)
cars_array = np.array(
    [
        [1, 170, 1, 0, 0, 4, 44.9, 0.2 * 44.9, -50 / 4, (60 / 170) * 44.9 * 0.25],
        [2, 230, 2, 0, 0, 5, 229, 0.2 * 229, -100 / 4, (60 / 230) * 229 * 0.25],
        [3, 230, 3, 0, 0, 3, 174, 0.2 * 174, -100 / 4, (60 / 230) * 174 * 0.25],
    ]
)
df = pd.DataFrame(
    cars_array,
    columns=[
        "Modelo",
        "Capacidade (km)",
        "ID",
        "Disponibilidade",
        "Viagem",
        "Capacidade",
        "Capacidade da bateria (kWh)",
        "Taxa de carga da bateria (kW)",
        "Taxa de descarga da bateria (kW)",
        "Taxa de carga específica (kW)",
    ],
)

df.head()

,Modelo,Capacidade (km),ID,Disponibilidade,Viagem,Capacidade,Capacidade da bateria (kWh),Taxa de carga da bateria (kW),Taxa de descarga da bateria (kW),Taxa de carga específica (kW)
0,1.0,170.0,1.0,0.0,0.0,4.0,44.9,8.98,-12.5,3.961765
1,2.0,230.0,2.0,0.0,0.0,5.0,229.0,45.80,-25.0,14.934783
2,3.0,230.0,3.0,0.0,0.0,3.0,174.0,34.80,-25.0,11.347826


In [632]:
cars = [
    {
        "autonomy_km": car[1],
        "autonomy_kwh": car[6],
        "tx_charge": car[7],
        "tx_discharge": np.abs(car[8]),
    }
    for car in cars_array
]

In [633]:
for car in cars:
    print(car)

{'autonomy_km': np.float64(170.0), 'autonomy_kwh': np.float64(44.9), 'tx_charge': np.float64(8.98), 'tx_discharge': np.float64(12.5)}
{'autonomy_km': np.float64(230.0), 'autonomy_kwh': np.float64(229.0), 'tx_charge': np.float64(45.800000000000004), 'tx_discharge': np.float64(25.0)}
{'autonomy_km': np.float64(230.0), 'autonomy_kwh': np.float64(174.0), 'tx_charge': np.float64(34.800000000000004), 'tx_discharge': np.float64(25.0)}


In [634]:
travels = [
    [(15, 20), (48, 55), (90, 92)],
    [(17, 24), (90, 92)],
    [(50, 55), (76, 89)],
]

In [635]:
def find_available_intervals(occupied_positions):
    available_intervals = []
    current_start = 0
    
    for start, end in sorted(occupied_positions):
        if current_start < start:
            available_intervals.append((current_start, start - 1))
        current_start = end + 1
    
    if current_start < NUM_INTERVALS:
        available_intervals.append((current_start, NUM_INTERVALS - 1))
    
    return available_intervals

In [636]:
available_intervals = [find_available_intervals(travel) for travel in travels]
available_intervals

[[(0, 14), (21, 47), (56, 89), (93, 95)],
 [(0, 16), (25, 89), (93, 95)],
 [(0, 49), (56, 75), (90, 95)]]

In [637]:
def calculate_time_to_charge_for_each_travel(
    travels, available_interval, tx_charge, tx_discharge
):
    spent_energy = [(end - start) * tx_discharge for start, end in travels]
    time_to_charge = np.ceil(spent_energy / tx_charge).astype(int)

    if len(travels) < len(available_interval):
        time_to_charge = np.append(time_to_charge, [0])

    return time_to_charge

In [638]:
time_to_charge = [
    calculate_time_to_charge_for_each_travel(
        travel, available_interval, car["tx_charge"], car["tx_discharge"]
    )
    for travel, available_interval, car in zip(
        travels.copy(), available_intervals.copy(), cars.copy()
    )
]

time_to_charge

[array([ 7, 10,  3,  0]), array([4, 2, 0]), array([ 4, 10,  0])]

In [639]:
def generate_min_zeros_random_vector(total_size, min_zeros):
    # Cria um vetor de zeros com tamanho N
    vector = np.zeros(min_zeros, dtype=int)

    vector = np.concatenate(
        (vector, np.random.choice((0, 1), size=(total_size - min_zeros)))
    )
    np.random.shuffle(vector)

    return vector

In [640]:
car_state_in_time = []

for available_slots, times in zip(available_intervals.copy(), time_to_charge.copy()):
    state_in_time = np.full(NUM_INTERVALS, -1)

    for slot, time in zip(available_slots.copy(), times.copy()):
        start, end = slot
        state_in_time[start : end + 1] = generate_min_zeros_random_vector(
            end - start + 1, time
        )

    car_state_in_time.append(state_in_time)

car_state_in_time

[array([ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  1, -1, -1,
        -1, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1, -1, -1, -1,
        -1, -1, -1, -1, -1,  0,  1,  1,  0,  1,  1,  0,  0,  0,  1,  0,  0,
         1,  0,  0,  1,  0,  1,  0,  1,  0,  1,  1,  0,  0,  0,  0,  0,  1,
         1,  1,  0,  1,  1, -1, -1, -1,  0,  1,  0]),
 array([ 0,  1,  1,  1,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  1,  1,  0,
        -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  0,  0,  1,  1,  0,  1,
         0,  1,  0,  1,  0,  1,  0,  1,  0,  0,  1,  1,  0,  0,  1,  0,  0,
         0,  1,  0,  0,  0,  1,  0,  1,  1,  0,  0,  1,  1,  1,  1,  0,  1,
         0,  1,  1,  1,  1,  0,  0,  1,  1,  0,  1,  1,  1,  0,  0,  0,  1,
         0,  0,  0,  0,  1, -1, -1, -1,  0,  0,  0]),
 array([ 1,  0,  0,  0,  0,  0,  1,  1,  0,  0,  1,  0,  1,  1,  0,  1,  1,
         1,  1,  1,  1,  0,  0,  1,  0,  1,  1,  0,  1, 